In [ ]:
import logging
import os
import urllib3

logging.captureWarnings(True)
http = urllib3.PoolManager()

repo_base = "https://raw.githubusercontent.com/earwole1/covid/master/"
files= ["covid.py", "population.py"]

for file in files:
  url = os.path.join(repo_base, file)
  r = http.request("GET", url)
  out_name = os.path.basename(url)
  with open(out_name, 'wb') as fp:
    fp.write(r.data)
    print(f"Copied {out_name}")


In [ ]:
from covid import Covid
c = Covid()

In [ ]:
# Find states with positive rate increasing faster than the testing rate over the last 28 days
worst_28 = c.find_worst_positive_to_testing(num_return=20, day_range=28)
c.print_worst_positive_to_testing(worst_28)

In [ ]:
_ = c.plot_all_by_date()

In [ ]:
ny = c.get_nyt_data()

In [ ]:
c.data = ny


In [ ]:
c.data = c.data.assign(hospitalizedCurrently=0)

In [ ]:
c.data = c.data.assign(positive=c.data.cases)


In [ ]:
c.data = c.data.assign(totalTestResults=0)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

dates = c.data.date.unique()
states = c.data.state.unique()
hosp = []
cases = []
deaths = []
data = c.data
for date in dates:
  temp = data[data.date == date]
  hosp_sum = temp.hospitalizedCurrently.sum()
  cases_sum = temp.cases.sum()
  deaths_sum = temp.deaths.sum()
  hosp.append(hosp_sum)
  cases.append(cases_sum)
  deaths.append(deaths_sum)


deaths = np.array(deaths)
cases = np.array(cases)
hosp = np.array(hosp)

smoothed_hosp = c.running_average(hosp, 7)
smoothed_deaths = c.running_average(deaths, 7)
smoothed_cases = c.running_average(cases, 7)

fig, ax = plt.subplots(3, 1)
fig.set_figwidth(12)
fig.set_figheight(19)
ax[0].grid()
ax[0].set_title("Covid Hospitalized - All States")
m = ax[0].plot(dates, hosp, 'b.', label='Hospitalized')
s = ax[0].plot(dates, smoothed_hosp, 'r-', lw=4, label='7-Day Average')
ax[0].legend()
ax[0].set_ylabel("Hospitalized")

ax[1].set_title("Hopitalization Rate")
ax[1].grid()
dr = ax[1].plot(dates, hosp / cases * 100, 'b.', label='Hosp. Rate %')
drs = ax[1].plot(dates, smoothed_hosp / smoothed_cases * 100, 'r-', lw=4, label='7-Day Average')
ax[1].legend()
ax[1].set_ylabel("Percentage")

ax[2].set_title("Death Rate")
ax[2].grid()
dr = ax[2].plot(dates, deaths / cases * 100, 'b.', label='Death Rate %')
drs = ax[2].plot(dates, smoothed_deaths / smoothed_cases * 100, 'r-', lw=4, label='7-Day Average')
ax[2].legend()
ax[2].set_ylabel("Percentage")




In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(12)
fig.set_figheight(12)
ax.set_yscale('log')
_ = ax.plot(dates, smoothed_cases, label='cases')
_ = ax.plot(dates, smoothed_hosp, label='hospitalized')
_ = ax.plot(dates, smoothed_deaths, label='deaths')
ax.grid()
ax.legend()

In [ ]:
states = ["Ohio", "Maryland", "Connecticut", "Colorado"]
for state in states:
  _ = c.plot_state(state, "cases")

In [ ]:
def plot_worst_states(metric: str, num_to_plot: int):
  worst = c.find_worst_states(metric, num_to_plot, Covid.DEFAULT_DAYS)
  for w in worst:
    state = w[0]
    try:
      _ = c.plot_state(state, metric)
    except ValueError as VE:
      print(f"Missing data for {state}.")

  for w in worst:
    state = w[0]
    tup = w[2][0]
    x, y, m = tup
    N = len(x)
    print(f"{state:20} {metric} daily change increasing at {m:7.1f}/day over last {N} days")

In [ ]:
plot_worst_states("cases", 4)

In [ ]:
plot_worst_states("hospitalizedCurrently", 4)

In [ ]:
plot_worst_states("positive", 4)

In [ ]:
c.pull_and_save("https://github.com/nytimes/covid-19-data/raw/master/rolling-averages/us-states.csv", "rolling.csv")

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
r = pd.read_csv("rolling.csv")
r.date = pd.to_datetime(r.date, format="%Y-%m-%d")

In [ ]:
oh = r[r.state == "Ohio"]
oh.sort_values("date", ascending=True, inplace=True)

In [ ]:
oh

In [ ]:
def plot_state(state):
  st = r[r.state == state]
  st.sort_values("date", ascending=True, inplace=True)
  fig, ax = plt.subplots()
  l1 = ax.plot(st.date, st.cases_avg, 'r-', label='Cases/Day')
  ax.grid()
  fig.set_figwidth(15)
  fig.set_figheight(10)
  ax.set_ylabel('Cases/Day')
  ax2 = ax.twinx()
  l2  = ax2.plot(st.date, st.deaths_avg, 'b--', label='Deaths/Day')
  ax2.set_ylabel('Deaths/Day')
  _ = ax.legend(loc='upper left')
  _ = ax2.legend(loc='lower right')
  ax.set_title(f"Per Day stats for {state}")

  fig, ax = plt.subplots()
  l1 = ax.plot(st.date, np.cumsum(st.cases_avg), 'r-', label='Total Cases')
  ax.grid()
  fig.set_figwidth(15)
  fig.set_figheight(10)
  ax.set_ylabel('Cases')
  ax2 = ax.twinx()
  l2  = ax2.plot(st.date, np.cumsum(st.deaths_avg), 'b--', label='Total Deaths')
  ax2.set_ylabel('Deaths')
  _ = ax.legend(loc='upper left')
  _ = ax2.legend(loc='lower right')
  ax.set_title(f"Cummukative stats for {state}")

  dr = np.cumsum(st.deaths_avg) / np.cumsum(st.cases_avg)
  dr[np.isinf(dr)] = 0
  dr *= 100.0 
  fig, ax = plt.subplots()
  _ = ax.plot(st.date, dr, 'k-', label='Death Rate')
  _ = ax.legend()
  fig.set_figwidth(15)
  fig.set_figheight(10)
  ax.grid()
  return st


In [ ]:
md = plot_state("Maryland")

In [ ]:
oh = plot_state("Ohio")

In [ ]:
fl = plot_state("Florida")

In [ ]:
states = r.state.unique()
tot = {}
for s in states:
  q = r[r.state == s]
  q.sort_values("date", ascending=True, inplace=True)
  q = q[-350:]
  _cases = q.cases_avg.sum()
  _deaths = q.deaths_avg.sum()
  if _cases == 0:
    dr = 0
  else:
    dr = _deaths / _cases * 100

  tot[s] = {"cases": q.cases_avg.sum(), "deaths": q.deaths_avg.sum(), "dr": dr}

In [ ]:
df = pd.DataFrame(tot).T
df.sort_values("dr", ascending=False, inplace=True)


In [ ]:
df